In [11]:
# Loading packages
import os
import nibabel as nib
import numpy as np
import subprocess
import pandas as pd
import time
import glob
from joblib import Parallel, delayed

# Directory where the data is stored - this should be for a single data type/label 
# (e.g. control data only, or other label data only)

directory = '/mnt/md0/cads-phd/For_SPR/super_mini_pipeline/'


In [12]:
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".nii") or filename.endswith(".nii.gz"): # Check for specific image file extensions
            img_path = os.path.join(folder_path, filename)
            img = nib.load(img_path)
            img_data = img.get_fdata()
            print(f"Shape of image {filename}: {img_data.shape}")
            images.append(img_data)
    return np.array(images)


loaded_images = load_images_from_folder(directory)
print(" You have loaded ", len(loaded_images), " images")


Shape of image sub-A00000456_ses-20090101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000368_ses-20110101_acq-mprage_run-01_T1w.nii: (192, 256, 256)
 You have loaded  2  images


In [13]:
#Let's apply FSL ANAT to the images


# n_jobs  = -1 # Number of jobs to run in parallel. -1 means use all available processors.
n_jobs = 10

def process_anat_volumes(directory, n_jobs=6):
    """
    Process anatomical volumes for files in a specified directory using parallel computing.

    Parameters:
    directory (str): The directory path where the files are located.
    n_jobs (int): The number of jobs to run in parallel (default is 6).

    Returns:
    None

    Example:
    process_anat_volumes('/path/to/your/directory/', n_jobs=8)
    """
    def anat_volumes(filename):
        full_path = os.path.join(directory, filename)
        return subprocess.run(["fsl_anat", "-i", full_path], capture_output=True)

    Parallel(n_jobs=n_jobs)(delayed(anat_volumes)(filename) for filename in os.listdir(directory))

process_anat_volumes(directory, n_jobs=n_jobs)


# took 35 mins to run 7 images on 10 cores
# took ~ 30 mins to process 2 images on 10 cores 

In [14]:
# Let's apply the warps to the images
def invwarp(directory):

    print(directory+"/T1.nii.gz")
    return subprocess.run(["invwarp", "--ref="+directory+"/T1.nii.gz", "--warp="+directory+"/T1_to_MNI_nonlin_field.nii.gz", "--out="+directory+"/Warps/"+"std_subject_space", "--verbose"], capture_output=True)

def applywarp_cort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)

def applywarp_subcort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-sub-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"subcort_HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)


# Apply warps to files in a specified directory in parallel
def apply_warps_parallel(directory, n_jobs=6):
    """
    Apply warps to files in a specified directory in parallel.

    Parameters:
    directory (str): The directory path where the files are located.
    n_jobs (int): The number of jobs to run in parallel (default is 6).

    Returns:
    None

    Example:
    apply_warps_parallel('/path/to/your/directory/', n_jobs=8)
    """
    def apply_warps(path):
        folder = os.path.join(path, "Warps") 
        if not os.path.exists(folder):
            os.mkdir(folder)

        invwarp(path)
        time.sleep(3)
        applywarp_cort(path)
        time.sleep(3)
        applywarp_subcort(path)
        time.sleep(3)

    Parallel(n_jobs=n_jobs)(delayed(apply_warps)(file) for file in glob.glob(os.path.join(directory, '*.anat')))


apply_warps_parallel(directory, n_jobs=15)

# takes 21 mins to run 7 images on 15 cores

/mnt/md0/cads-phd/For_SPR/super_mini_pipeline/sub-A00000368_ses-20110101_acq-mprage_run-01_T1w.anat/T1.nii.gz
/mnt/md0/cads-phd/For_SPR/super_mini_pipeline/sub-A00000456_ses-20090101_acq-mprage_run-02_T1w.anat/T1.nii.gz


In [72]:
# Here we generate, extract and build the dataframe for the volumes of the subcortical and cortical regions

subcort_path = 'HarvardOxford_Subcortical.csv'
cort_path = 'HarvardOxford_Cortical.csv'
label = 'SPR'  # Not sure if you want this as a separate column or just a variable

subcort_atlas = pd.read_csv(subcort_path)  # Suppose it has a column "Label" in the correct order
cort_atlas = pd.read_csv(cort_path)        # Suppose it has a column "Label" in the correct order

directory = '/mnt/md0/cads-phd/For_SPR/super_mini_pipeline/'
# directory = "/some/path/with/anat/files"   # or however you define it

# A list to hold the row dictionaries for each subject
all_rows = []

for filename in os.listdir(directory):
    if filename.endswith(".anat"):
        img_path = os.path.join(directory, filename)
        print("img_path:", img_path)
        print("filename:", filename)
        
        # Derive the patient ID from the filename (e.g., removing '.anat')
        patient = filename[:-5]  # or whatever logic you need
        
        # -------------------
        # 1) CORTICAL STATS
        # -------------------
        out_cort = subprocess.Popen(
            [
                "fslstats", 
                "-K", img_path + "/Warps/HO_in_subj_t1_space.nii.gz", 
                img_path + "/T1.nii.gz", 
                "-V"
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT
        )
        stdout_cort, stderr_cort = out_cort.communicate()
        
        cort_df = stdout_cort.decode("utf-8")
        df_cort = pd.DataFrame(
            cort_df.replace('\n', ',').split(","), 
            columns=['vol']
        )
        df_cort['vol'] = df_cort['vol'].str.replace(',', '', regex=True)
        df_cort[['Voxel', 'Volume']] = df_cort['vol'].str.split(' ', 1, expand=True)
        df_cort = df_cort.drop(columns='vol')
        
        # -------------------
        # 2) SUBCORT STATS
        # -------------------
        process = subprocess.Popen(
            [
                "fslstats",
                "-K", img_path + "/Warps/subcort_HO_in_subj_t1_space.nii.gz",
                img_path + "/T1.nii.gz",
                "-V"
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT
        )
        stdout_subcort, stderr_subcort = process.communicate()
        
        subcort_df = stdout_subcort.decode("utf-8")
        df_subcort = pd.DataFrame(
            subcort_df.replace('\n', ',').split(","), 
            columns=['vol']
        )
        df_subcort['vol'] = df_subcort['vol'].str.replace(',', '', regex=True)
        df_subcort[['Voxel', 'Volume']] = df_subcort['vol'].str.split(' ', 1, expand=True)
        df_subcort = df_subcort.drop(columns='vol')
        
        # ----------------------------
        # 3) BUILD DICTIONARY FOR ROW
        # ----------------------------
        # Ensure the lengths match expected # of labels; 
        # the order in subcort_atlas['Label'] matches df_subcort['Volume'] row by row
        subcort_dict = dict(zip(subcort_atlas['Label'], df_subcort['Volume']))
        cort_dict    = dict(zip(cort_atlas['Label'],   df_cort['Volume']))
        
        # Merge them into a single dict
        row_dict = {**subcort_dict, **cort_dict}
        
        # Add additional columns
        row_dict["Patient"] = patient
        row_dict["Target"]  = label  # e.g., "SPR", or rename to "target" etc.
        
        # ---------------------------
        # 4) APPEND to all_rows list
        # ---------------------------
        all_rows.append(row_dict)

# ------------------------------------------
# 5) BUILD A FINAL DATAFRAME FROM all_rows
# ------------------------------------------
volume_df = pd.DataFrame(all_rows)

# If you want specific ordering of columns,
# define the order you want:
# e.g. columns for patient, your label, then subcort, then cort
ordered_cols = (
    ["Patient", "Target"] 
    + subcort_atlas["Label"].tolist()
    + cort_atlas["Label"].tolist()
)
# Reindex volume_df to enforce that order (only if you want to be strict):
volume_df = volume_df.reindex(columns=ordered_cols)

print("\nFINAL DATAFRAME:\n")
volume_df

img_path: /mnt/md0/cads-phd/For_SPR/super_mini_pipeline/sub-A00000368_ses-20110101_acq-mprage_run-01_T1w.anat
filename: sub-A00000368_ses-20110101_acq-mprage_run-01_T1w.anat
img_path: /mnt/md0/cads-phd/For_SPR/super_mini_pipeline/sub-A00000456_ses-20090101_acq-mprage_run-02_T1w.anat
filename: sub-A00000456_ses-20090101_acq-mprage_run-02_T1w.anat

FINAL DATAFRAME:
                                             Patient Target  \
0  sub-A00000368_ses-20110101_acq-mprage_run-01_T1w    SPR   
1  sub-A00000456_ses-20090101_acq-mprage_run-02_T1w    SPR   

  Left Cerebral White Matter Left Cerebral Cortex  Left Lateral Ventricle  \
0             157016.149742         346764.330700            6727.006415    
1             161196.980019         342422.081802            7401.044996    

  Left Thalamus  Left Caudate  Left Putamen Left Pallidum     Brain-Stem  ...  \
0  5333.005086   3685.003514   5652.005390   1432.001366   22943.021880   ...   
1  6192.037645   2315.014074   3860.023468   1530.00

In [73]:
# Save the dataframe to a CSV file
# volume_df.to_csv("volume_data.csv", index=False)  # uncomment to save volumetric data to a CSV file

In [71]:
# We will now build the final dataframe by merging the processed and extracted volume data and merging it with the original data containing age and sex. 

original = pd.read_csv('CN_SPR_originalData.csv')
original

final_df = pd.merge(volume_df, original, on="Patient")
final_df = final_df.set_index('Patient')  # index on Patient 
to_drop = ['dx', 'Z max', 'center X', 'cener Y', 'study', 'participant_id']
final_df = final_df.drop(columns=to_drop)
final_df

,Target,Left Cerebral White Matter,Left Cerebral Cortex,Left Lateral Ventricle,Left Thalamus,Left Caudate,Left Putamen,Left Pallidum,Brain-Stem,Left Hippocampus,...,Frontal Operculum Cortex,Central Opercular Cortex,Parietal Operculum Cortex,Planum Polare,Heschl's Gyrus (includes H1 and H2),Planum Temporale,Supracalcarine Cortex,Occipital Pole,age,sex
Patient,,,,,,,,,,,,,,,,,,,,,
sub-A00000368_ses-20110101_acq-mprage_run-01_T1w,SPR,157016.149742,346764.330700,6727.006415,5333.005086,3685.003514,5652.005390,1432.001366,22943.021880,2890.002756,...,1616.001541,8671.008269,2551.002433,2871.002738,1082.001032,1289.001229,134.000128,14742.014059,52,male
sub-A00000456_ses-20090101_acq-mprage_run-02_T1w,SPR,161196.980019,342422.081802,7401.044996,6192.037645,2315.014074,3860.023468,1530.009302,25591.155585,3059.018598,...,1518.009229,7280.044260,2115.012859,1867.011351,914.005557,1055.006414,144.000875,13825.084051,53,male


In [74]:
# Save the final dataframe to a CSV file
# final_df.to_csv("final_data.csv", index=True)  # uncomment to save the final dataframe to a CSV file

In [ ]:
# Please see the ------ for the model building code.
